In [21]:
!pip install tpulse --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tpulse import TinkoffPulse # основная библиотека для парсинга
from httpx import HTTPStatusError
from time import sleep


pulse = TinkoffPulse()
def parse_tickers(tickers):
    '''
    tickers: list of str
    '''
    for ticker in tickers:
        N = 300 # параметр, контролирующий глубину
        cursor = 999999999
        raw_data = []
        
        KEYS = [
            "id",
            "nickname",
            "commentsCount",
            "inserted",
            "reactions",
            "text"
        ]
        
        for _ in range(N):
            try:
                response = pulse.get_posts_by_ticker(ticker, cursor)
                if (response == None):
                    continue
                cursor = response["nextCursor"]
                posts = response["items"]
                for post in posts:
                    data = {
                        key: post[key] for key in KEYS if key != "nickname" and key != "text" and key != "reactions"
                    }
                    data["nickname"] = post["owner"]["nickname"]
                    data["text"] = post["content"]["text"]
                    data["reactions"] = post["reactions"]["totalCount"]
                    raw_data.append(data)
            except HTTPStatusError:
                print("Error for", ticker)
                continue
            except ReadTimeout:
                print("Error for", ticker)
                continue
            sleep(0.25)
        pd.DataFrame(raw_data).to_csv(f"{ticker}_comments.csv")


In [49]:
# парсил отдельно, поэтому здесь сохранилась только часть
parse_tickers(["LKOH", "SNGS", "CHMF"])